In [2]:
import requests
from bs4 import BeautifulSoup as soup
import random
import time
from datetime import datetime
from dateutil.parser import parse
import sys

In [3]:
def GetNextPage(content):
    next_page = content.find('div',{'class':'btn-group btn-group-paging'}).find_all('a',{'class':'btn wide'})
    next_page_url = next_page[1].attrs['href']
    page_number = next_page_url.split('index')[1].split('.')[0]
    return page_number

In [4]:
def LoadUserAgentFile(file):
    agent_list = []
    with open(file, 'r+') as f:
        agents = f.readlines()
    for agent in agents:
        agent_list.append(agent.replace('\n',''))
    return agent_list

In [5]:
def GetUserAgent():
    index = int(random.random()* len(agent_list))
    headers = {'User-Agent':agent_list[index]}
    return headers

In [6]:
def GetUrl(content):
    page_url = content.find('a').attrs['href']
    return page_url

In [8]:
def GetPageContent(url, headers):
    po_cont = {}
    req = requests.get(url, headers = headers)
    tree = soup(req.text, 'lxml')
    meta = tree.find_all('span',{'class':'article-meta-value'})
    cont = req.text.split()
    pushes = tree.find_all('span',{'class':'f3 push-content'})
    push_list = []
    for push in pushes:
        push_list.append(push.text.replace(': ',''))
    po_cont['push'] = push_list
    po_cont['author'] = meta[0].text
    po_cont['title'] = meta[2].text
    po_cont['time'] = meta[3].text
    cont = req.text.split(meta[3].text)[1].split('--')[0].replace('\n',' ').replace('</span></div>','')
    po_cont['cont'] = cont
    return po_cont

In [27]:
def CheckTime(day):
    d = datetime.strptime(day, '%Y%m%d')
    t_d = '{}/{}'.format(d.month, d.day)
    return t_d

In [60]:
def GetPostInTime(day, url):
    target_page_list = []
    divs = soup(requests.get(url, GetUserAgent()).text, 'lxml').find_all('div',{'class':'r-ent'})
    for div in divs:
        try:
            po_time = div.find('div',{'class':'date'}).text.strip()
            link = div.find('div',{'class':'title'}).find('a').attrs['href']
            if(po_time == day):
                target_page_list.append(link)
        except:
            continue
    return target_page_list

In [61]:
def GetDaily(day, page_number, url, file, board):
    total_page_list = []
    for num in range(int(page_number)+1,1,-1):
        page_url = url+'/bbs/'+board+'/index'+str(num)+'.html'
        target_page_list = GetPostInTime(day, page_url)
        if(len(target_page_list) > 0):
            for target in target_page_list:
                total_page_list.append(target)
        else:
            break
    with open(file, 'a+', encoding = 'utf8') as f:
        for page in total_page_list:    
            detail = GetPageContent(url+page, GetUserAgent())
            f.write(str(detail)+'\n')
        f.flush()

In [62]:
url = 'https://www.ptt.cc'
board_name = 'GirlsFront'
target_day = '20180516'
output_file = '{}_{}.csv'.format(board_name, target_day)
first_page = url+'/bbs/'+board_name+'/index.html'
agent_list = LoadUserAgentFile('useragent_list.txt')
results = requests.get(first_page, GetUserAgent())
content = soup(results.text, 'lxml')
page_number= GetNextPage(content)
day = CheckTime(target_day)

In [63]:
GetDaily(day, page_number, url, output_file, board_name)